### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch
import torch.nn as nn

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size
import time


handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
handler args: ('NodeEncoderBatchNorm1d',)
handler args: ('EdgeEncoderBatchNorm1d',)


### PyTorch Model

In [2]:
torch_model = InteractionNetwork(flow="source_to_target", out_channels=128).eval() # eval mode for bathnorm

# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)
# new_state_dict = OrderedDict()
# for key in torch_model.O.layers[1].state_dict():
# #     print(torch_model.O.layers[1].state_dict()[key].shape)
#     if key != "num_batches_tracked":
#         new_state_dict[key] = torch.rand(40)

# torch_model.O.layers[1].load_state_dict(new_state_dict)
# print(torch_model.O.layers[1].state_dict())
# for name, submodule in torch_model.named_modules():
#     if name != "":
#         print(f"{name}: {submodule}")

In [3]:
state_dict = torch.load('./Tau3MuGNNs/data/logs/10_01_2022_22_20_47-GNN_full_dR_1/model.pt', map_location="cpu")
state_dict

{'epoch': 310,
 'model_state_dict': OrderedDict([('convs.0.t', tensor([3.85888600])),
              ('convs.0.p', tensor([1.])),
              ('convs.1.t', tensor([1.09752059])),
              ('convs.1.p', tensor([1.])),
              ('convs.2.t', tensor([1.65537775])),
              ('convs.2.p', tensor([1.])),
              ('convs.3.t', tensor([1.92783535])),
              ('convs.3.p', tensor([1.])),
              ('convs.4.t', tensor([1.64153087])),
              ('convs.4.p', tensor([1.])),
              ('convs.5.t', tensor([1.49196327])),
              ('convs.5.p', tensor([1.])),
              ('convs.6.t', tensor([1.45402455])),
              ('convs.6.p', tensor([1.])),
              ('convs.7.t', tensor([1.18432832])),
              ('convs.7.p', tensor([1.])),
              ('mlps.0.0.weight',
               tensor([[-0.13923848, -0.12548204,  0.22906402,  ..., -0.74014211,
                        -0.09414767, -0.24523829],
                       [ 0.00451932, -0.388364

In [4]:
torch_model.beta = state_dict['model_state_dict']['convs.0.t']
# print(type(torch_model.node_encoder.weight))
# print(type(siqi_model_state_dict['model_state_dict']['node_encoder.weight']))
torch_model.node_encoder.weight = nn.Parameter(state_dict['model_state_dict']['node_encoder.weight'])
torch_model.node_encoder.bias = nn.Parameter(state_dict['model_state_dict']['node_encoder.bias'])
torch_model.edge_encoder.weight = nn.Parameter(state_dict['model_state_dict']['edge_encoder.weight'])
torch_model.edge_encoder.bias = nn.Parameter(state_dict['model_state_dict']['edge_encoder.bias'])
torch_model.node_encoder_norm.weight = nn.Parameter(
    state_dict['model_state_dict']['bn_node_feature.weight']
)
torch_model.node_encoder_norm.norm.weight = torch_model.node_encoder_norm.weight # this is temporary soln to the structure of the class

torch_model.node_encoder_norm.bias = nn.Parameter(
    state_dict['model_state_dict']['bn_node_feature.bias']
)
torch_model.node_encoder_norm.norm.bias = torch_model.node_encoder_norm.bias # this is temporary soln to the structure of the class

torch_model.node_encoder_norm.running_mean = nn.Parameter(
    state_dict['model_state_dict']['bn_node_feature.running_mean']
)
torch_model.node_encoder_norm.norm.running_mean = torch_model.node_encoder_norm.running_mean # this is temporary soln to the structure of the class

torch_model.node_encoder_norm.running_var = nn.Parameter(
    state_dict['model_state_dict']['bn_node_feature.running_var']
)
torch_model.node_encoder_norm.norm.running_var = torch_model.node_encoder_norm.running_var # this is temporary soln to the structure of the class


torch_model.edge_encoder_norm.weight = nn.Parameter(
    state_dict['model_state_dict']['bn_edge_feature.weight']
)
torch_model.edge_encoder_norm.norm.weight = torch_model.edge_encoder_norm.weight # this is temporary soln to the structure of the class

torch_model.edge_encoder_norm.bias = nn.Parameter(
    state_dict['model_state_dict']['bn_edge_feature.bias']
)
torch_model.edge_encoder_norm.norm.bias = torch_model.edge_encoder_norm.bias # this is temporary soln to the structure of the class

torch_model.edge_encoder_norm.running_mean = nn.Parameter(
    state_dict['model_state_dict']['bn_edge_feature.running_mean']
)
torch_model.edge_encoder_norm.norm.running_mean = torch_model.edge_encoder_norm.running_mean # this is temporary soln to the structure of the class

torch_model.edge_encoder_norm.running_var = nn.Parameter(
    state_dict['model_state_dict']['bn_edge_feature.running_var']
)
torch_model.edge_encoder_norm.norm.running_var = torch_model.edge_encoder_norm.running_var # this is temporary soln to the structure of the class

mlp_name = "mlps.0."
original_layer_idxs = [0,1,4] # don't ask me why it jumps from 1 to 4
new_layer_mlp_idxs = [0,1,3] # we skip 2 bc that's activation
for idx in range(len(original_layer_idxs)):
    original_layer_idx = original_layer_idxs[idx]
    new_layer_mlp_idx = new_layer_mlp_idxs[idx]
    module = torch_model.O.layers[new_layer_mlp_idx]
#     torch_model.O.layers[idx] = nn.Parameter(
#         state_dict['model_state_dict'][f""]
#     )
    if (module.__class__.__name__ == 'Linear') or (module.__class__.__name__ == 'BatchNorm1d'):
        module.weight = nn.Parameter(
            state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.weight"]
        )
        module.bias = nn.Parameter(
            state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.bias"]
        )
    if (module.__class__.__name__ == 'BatchNorm1d'):
#         print('BatchNorm1d')
        module.running_mean = nn.Parameter(
            state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.running_mean"]
        )
        module.running_var = nn.Parameter(
            state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.running_var"]
        )
        
        

In [5]:
feat_l = ["weight", "bias", "running_mean", "running_var"]
for feat in feat_l:
    bool_val = torch.all(
        torch_model.node_encoder_norm.state_dict()[feat] == state_dict['model_state_dict']['bn_node_feature.'+feat]
    )
    print(bool_val)
    bool_val = torch.all(
        torch_model.edge_encoder_norm.state_dict()[feat] == state_dict['model_state_dict']['bn_edge_feature.'+feat]
    )
    print(bool_val)



tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


In [6]:
new_layer_mlp_idxs = [0,1,3] 
for idx in new_layer_mlp_idxs:
    print(idx)
    print(torch_model.O.layers[idx].state_dict())

0
OrderedDict([('weight', tensor([[-0.13923848, -0.12548204,  0.22906402,  ..., -0.74014211,
         -0.09414767, -0.24523829],
        [ 0.00451932, -0.38836420,  0.20088144,  ..., -0.20723528,
         -0.04415414,  0.18898520],
        [ 0.07053071, -0.23893164, -0.01400635,  ..., -0.56174874,
          0.35805658,  0.23247291],
        ...,
        [-0.01958065,  0.07542589, -0.00582928,  ..., -0.12086138,
          0.02530678,  0.05078236],
        [ 0.09340435,  0.05330789, -0.14644164,  ..., -0.75741583,
          0.50431025,  0.33890775],
        [ 0.01767002, -0.22854996, -0.06237415,  ...,  0.00471479,
         -0.32891613,  0.30723211]])), ('bias', tensor([ 1.65516642e-04,  1.16245134e-03, -1.01977552e-03,  2.92035780e-04,
        -4.49627842e-04,  4.02447884e-04, -4.15552058e-04,  1.80136703e-03,
         1.60019065e-03, -7.43212528e-04, -4.10989771e-04, -4.19527714e-05,
         7.41289579e-04, -2.27678742e-04,  8.05305433e-04, -1.13270921e-03,
         1.31709021e-04,  1

In [7]:
# torch.all(torch_model.node_encoder.state_dict()["weight"] == siqi_model_state_dict['model_state_dict']['node_encoder.weight'])
# torch.all(torch_model.node_encoder.state_dict()["bias"] == siqi_model_state_dict['model_state_dict']['node_encoder.bias'])
idx=2
original_layer_idx = original_layer_idxs[idx]
new_layer_mlp_idx = new_layer_mlp_idxs[idx]
boolean_val = torch.all(
    torch_model.O.layers[new_layer_mlp_idx].state_dict()["weight"] == state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.weight"]
)
print(boolean_val)


boolean_val = torch.all(
    torch_model.O.layers[new_layer_mlp_idx].state_dict()["bias"] == state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.bias"]
)
print(boolean_val)

idx=1
original_layer_idx = original_layer_idxs[idx]
new_layer_mlp_idx = new_layer_mlp_idxs[idx]
boolean_val = torch.all(
    torch_model.O.layers[new_layer_mlp_idx].state_dict()["running_mean"] == state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.running_mean"]
)
print(boolean_val)


boolean_val = torch.all(
    torch_model.O.layers[new_layer_mlp_idx].state_dict()["running_var"] == state_dict['model_state_dict'][mlp_name+f"{original_layer_idx}.running_var"]
)
print(boolean_val)



tensor(True)
tensor(True)
tensor(True)
tensor(True)


In [8]:
torch_model.O.layers[3]

Linear(in_features=256, out_features=128, bias=True)

### HLS Model

In [9]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
forward_dict["node_encoder_norm"] = "NodeEncoderBatchNorm1d"
forward_dict["edge_encoder_norm"] = "EdgeEncoderBatchNorm1d"
# forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
# forward_dict["R2"] = "EdgeBlock"

In [10]:
common_dim = 128 #5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_attr": 3,
        "node_dim": common_dim,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [11]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [12]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<52,20>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid', #sigmoid
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
pyg_to_hls config: {'OutputDir': 'test_GNN', 'ProjectName': 'myproject', 'Backend': 'Vivado', 'XilinxPart': 'xcku115-flvb2104-2-i', 'Board': None, 'ClockPeriod': 5, 'IOType': 'io_parallel', 'HLSConfig': {'Model': {'Precision': 'ap_fixed<52,20>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}, 'PytorchModel': InteractionNetwork(), 'InputShape': {'NodeAttr': [28, 3], 'EdgeAttr': [37, 4], 'EdgeIndex': [37, 2], 'NodeDim': 128, 'EdgeDim': 128}, 'ForwardDictionary': OrderedDict([('node_encoder', 'NodeEncoder'), ('edge_encoder', 'EdgeEncoder'), ('node_encoder_norm', 'NodeEncoderBatchNorm1d'), ('edge_encoder_norm', 'EdgeEncoderBatchNorm1d'), ('O', 'NodeBlock')]), 'ActivateFinal': 'sigmoid', 'ParallelizationFactor': 16, 'gnn_resource_limit': 'false'}
PygModelReader node_dim: 128
PygModelReader edge_dim: 128
PygModelReader node_attr: 3
PygModel

## hls_model.compile() builds the C-function for the model

In [13]:
hls_model.compile()

Writing HLS project
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
def_cpp: layer5_t layer5_out[N_LAYER_1_5*N_LAYER_2_5]
def_cpp: layer6_t layer6_out[N_LAYER_1_4*N_LAYER_2_4]
def_cpp: layer7_t layer7_out[N_LAYER_1_5*N_LAYER_2_5]
def_cpp: layer8_t layer8_out[N_NODE*LAYER8_OUT_DIM]
def_cpp: layer9_t layer9_out[N_LAYER_1_4*LAYER9_OUT_DIM]
Done
lib_name: firmware/myproject-f6446afd.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [14]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [15]:
# data = graphs[0]
# # print(f"graphs len: {len(graphs)}")
# print(type(data.hls_data))
# # print(f"data.x shape:{data.x.shape}")
# torch_pred = torch_model(data)
# torch_pred = torch_pred.detach().cpu().numpy().flatten()
# # print(f"torch pred shape: {torch_pred.shape}")
# hls_pred = hls_model.predict(data.hls_data)
# # time.sleep(9)
# print(f"hls_pred.shape: {hls_pred.shape}")
# MSE = mean_squared_error(torch_pred, hls_pred)
# print(f"MSE: {MSE}")
# # print((np.abs(torch_pred- hls_pred)))
# # print(torch_pred)
# # print(hls_pred)

In [16]:
# print(data.edge_index.shape)
# print(data.hls_data[-1].shape)

In [17]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [18]:
# MSE_l = []
# for data in graphs:
# #     print(f"graphs len: {len(graphs)}")
# #     print(type(data.hls_data))
# #     print(f"data.x shape:{data.x.shape}")
#     torch_pred = torch_model(data)
#     torch_pred = torch_pred.detach().cpu().numpy().flatten()
# #     print(f"torch pred shape: {torch_pred.shape}")
#     hls_pred = hls_model.predict(data.hls_data)
# #     print(f"hls_pred.shape: {hls_pred.shape}")
#     MSE = mean_squared_error(torch_pred, hls_pred)
# #     get_meanNstd(data, torch_pred, hls_pred)
# #     print(f"MSE: {MSE}")
#     MSE_l.append(MSE)
# #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
# #     print(f"MAPE: {MAPE}")

# MSE_l = np.array(MSE_l)
# print(f"MSE_l: {MSE_l}")
# print(f"overall MSE: {np.mean(MSE_l)}")
# print(MSE_l.shape)

In [19]:
# for data_instance in data.hls_data:
#     print(data_instance.shape)
    
# print(data.edge_index.shape)

In [20]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =3000): #n_graphs =1000
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array

        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        else:
            print("bad graph")
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [21]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7ff89007c910>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7ff8909cbe50>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7ff890094f50>}


In [22]:
# stage = "train"
# graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
# data = graphs[1]
# with open('test_data.pickle', 'wb') as f:
#     pkl.dump(data, f)

In [23]:
with open('test_data.pickle', 'rb') as f:
    data= pkl.load(f) 

print(f"data.x.shape: {data.x.shape}")
# print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
# print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
# print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
# time.sleep(9)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")
# print((np.abs(torch_pred- hls_pred)))
# print(torch_pred)
# print(hls_pred)

data.x.shape: torch.Size([28, 3])
<class 'list'>
hls_pred.shape: (3584,)
MSE: 4.799802582056145e-07


In [24]:
# from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
# node_scaler = StandardScaler()
# edge_scaler = StandardScaler()
# for stage in [stages[0]]:
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
#     print(type(graphs))

#     for data in graphs: # fit data to scaler b4 feedforwarding
# #         print(f"data type: {type(data)}")
#         node_scaler.fit(data.x) 
#         edge_scaler.fit(data.edge_attr) 
    counter = 0
    for data in graphs:
       
#         data.x = torch.from_numpy(node_scaler.transform(data.x)).float()
#         data.edge_attr = torch.from_numpy(edge_scaler.transform(data.edge_attr)).float()
#         data.hls_data[0] = node_scaler.transform(data.hls_data[0])
#         data.hls_data[1] = edge_scaler.transform(data.hls_data[1])
        if counter%1000 ==0:
            print(f"counter: {counter}")
        counter += 1
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
#         print(f"torch_pred shape: {torch_pred.shape}")
        hls_pred = hls_model.predict(data.hls_data)
#         print(f"hls_pred shape: {hls_pred.shape}")
        MSE = mean_squared_error(torch_pred, hls_pred)
#         print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
#         break
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")
print(f"MSE max: {np.max(MSE_l)}")

n_graphs: 3000
writing test bench data for 1st graph
counter: 0
counter: 1000
counter: 2000
n_graphs: 3000
writing test bench data for 1st graph
counter: 0
counter: 1000
counter: 2000
n_graphs: 3000
writing test bench data for 1st graph
counter: 0
counter: 1000
counter: 2000
MSE_l :[4.8035707e-07 4.8258255e-07 3.9513739e-07 ... 4.1353209e-07 3.4346505e-07
 4.2534481e-07]
MSE means: 6.07427750765055e-07
MSE max: 0.0017039751401171088


In [25]:
MSE_l.shape

(9000,)

In [26]:
# torch_model.state_dict()
with open('MSE_l.npy', 'wb') as f:
    np.save(f, MSE_l)